# Instalação de bibliotecas

In [2]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [3]:
 pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

Abrindo o dataframe já pré - processado, ou seja, que passou pela limpeza, tokenização e remoção de stop words.

In [6]:
pre_processing = pd.read_csv('../output/pre_processamento.csv')

# Aplicação Bag of Words

Durante esta etapa é feita a implementação do modelo Bag of Words, que vetoriza as palavras e faz a contagem de vezes que cada uma aparece na coluna a ser utilizada. Assim, tendo como resultado um dataframe com as palavras que mais aparecem no texto e sua respectiva frequência.

In [7]:
def bow_dataframe(inputUser):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(inputUser)
    # Com essa variavel é possível ver as palavras e contar sua frequência
    wordCount = tokenizer.word_counts    
    dfCountBoW = pd.DataFrame(list(wordCount.items()))
    dfCountBoW.rename(columns={0: "Palavra", 1:"Frequência"}, inplace=True)
    final_df = dfCountBoW.sort_values(by=['Frequência'], ascending=False)
    return final_df

#### Caso de teste

In [19]:
# Testando a função 
bow_test = pre_processing
bow_test_result = bow_dataframe(bow_test['texto'])

In [20]:
# Input da função
pre_processing

,texto
0,"['minuto', 'touro', 'ouro']"
1,['sim']
2,"['queria', 'saber', 'banking', 'própria']"
3,[]
4,[]
...,...
6351,"['excelente', 'explicação']"
6352,"['porque', 'brg', 'tá', 'diminuído', 'limite',..."
6353,"['atendam', 'telefone', 'amor', 'deus']"
6354,"['grato', 'todo', 'esforço', 'rendeu', 'invest..."


In [15]:
# Output da função
bow_test_result.head(5)

,Palavra,Frequência
0,texto,1


#### Demonstração da função

Nesta etapa será demonstrado o funcionamento da função na base de dados real 

In [18]:
result_output = bow_dataframe(pre_processing['texto'])
result_output.head(10)

,Palavra,Frequência
93,'banco',475
583,'limite',363
15,'conta',319
294,'cartão',310
183,'melhor',228
144,'agora',218
30,'sempre',181
230,'fazer',172
32,'vai',172
189,'dinheiro',168
